In [42]:
import os, sys
import numpy as np
import pandas as pd
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
import seaborn as sns
import matplotlib.pyplot as plt
import random

from pylab import rcParams
from torch.utils.data import Dataset, DataLoader
from matplotlib import rc
from matplotlib.ticker import MaxNLocator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics.functional import accuracy
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

rcParams['figure.figsize'] = 16, 10

In [3]:
pl.seed_everything(42)

Global seed set to 42


42

In [4]:
X_test, y_test = pd.read_csv('test_dataset.csv', index_col=False), pd.read_csv('test_labels.csv', index_col=False)
X_train, y_train = pd.read_csv('train_dataset.csv', index_col=False), pd.read_csv('train_labels.csv', index_col=False)    

In [4]:
X, y = pd.read_csv("data_alco.csv", index_col=False), pd.read_csv("labels_alco.csv", index_col=False)

C:\Users\Kuban\.conda\envs\master\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [8]:
y.head()

,0,1
0,0,a
1,1,a
2,2,a
3,3,a
4,4,a


In [20]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,0,0.000000,-8.921,0.834,-19.847,8.148,-2.146,1.129,-0.071,3.408,...,3.906,5.157,1.048,-6.266,-9.033,-2.421,-4.313,-0.478,-8.901,-5.636
1,1,0.003906,-8.433,3.276,-12.522,1.801,-2.146,0.641,-0.559,1.455,...,10.254,7.111,0.559,-8.708,-12.451,-3.886,-5.290,-0.966,-7.924,-2.706
2,2,0.007812,-2.574,5.717,1.149,-2.594,-1.658,-0.336,-1.048,0.478,...,1.465,-2.655,0.559,-9.196,-11.963,-4.862,-5.290,-0.966,-3.042,1.689
3,3,0.011719,5.239,7.670,14.821,-4.547,-0.682,-0.824,-0.559,0.966,...,5.371,-7.050,0.559,-6.755,-8.545,-3.886,-4.313,-0.966,4.771,5.595
4,4,0.015625,11.587,9.623,20.681,-5.035,2.248,0.641,0.905,1.943,...,-12.207,-4.120,1.048,-3.337,-3.662,-1.933,-2.360,-0.478,11.607,9.013


# PREPROCESSING

In [139]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y['1'])


In [140]:
encoded_labels

array([0, 0, 0, ..., 1, 1, 1])

In [141]:
label_encoder.classes_

array(['a', 'c'], dtype=object)

In [12]:
y['label'] = encoded_labels

In [14]:
y.head()

,0,1,label
0,0,a,0
1,1,a,0
2,2,a,0
3,3,a,0
4,4,a,0


In [119]:
X.head()

,2,3,4,5,6,7,8,9,10,11,...,56,57,58,59,60,61,62,63,64,65
0,-8.921,0.834,-19.847,8.148,-2.146,1.129,-0.071,3.408,-0.092,4.832,...,3.906,5.157,1.048,-6.266,-9.033,-2.421,-4.313,-0.478,-8.901,-5.636
1,-8.433,3.276,-12.522,1.801,-2.146,0.641,-0.559,1.455,0.397,6.297,...,10.254,7.111,0.559,-8.708,-12.451,-3.886,-5.290,-0.966,-7.924,-2.706
2,-2.574,5.717,1.149,-2.594,-1.658,-0.336,-1.048,0.478,-1.068,5.809,...,1.465,-2.655,0.559,-9.196,-11.963,-4.862,-5.290,-0.966,-3.042,1.689
3,5.239,7.670,14.821,-4.547,-0.682,-0.824,-0.559,0.966,-3.510,3.367,...,5.371,-7.050,0.559,-6.755,-8.545,-3.886,-4.313,-0.966,4.771,5.595
4,11.587,9.623,20.681,-5.035,2.248,0.641,0.905,1.943,-5.463,1.414,...,-12.207,-4.120,1.048,-3.337,-3.662,-1.933,-2.360,-0.478,11.607,9.013


In [118]:
FEATURE_COLUMS = X.columns.tolist()[2:]
FEATURE_COLUMS

['4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65']

In [23]:
len(X)/256 == len(y)

True

In [24]:
X = X[FEATURE_COLUMS]

In [25]:
X.head()

,2,3,4,5,6,7,8,9,10,11,...,56,57,58,59,60,61,62,63,64,65
0,-8.921,0.834,-19.847,8.148,-2.146,1.129,-0.071,3.408,-0.092,4.832,...,3.906,5.157,1.048,-6.266,-9.033,-2.421,-4.313,-0.478,-8.901,-5.636
1,-8.433,3.276,-12.522,1.801,-2.146,0.641,-0.559,1.455,0.397,6.297,...,10.254,7.111,0.559,-8.708,-12.451,-3.886,-5.290,-0.966,-7.924,-2.706
2,-2.574,5.717,1.149,-2.594,-1.658,-0.336,-1.048,0.478,-1.068,5.809,...,1.465,-2.655,0.559,-9.196,-11.963,-4.862,-5.290,-0.966,-3.042,1.689
3,5.239,7.670,14.821,-4.547,-0.682,-0.824,-0.559,0.966,-3.510,3.367,...,5.371,-7.050,0.559,-6.755,-8.545,-3.886,-4.313,-0.966,4.771,5.595
4,11.587,9.623,20.681,-5.035,2.248,0.641,0.905,1.943,-5.463,1.414,...,-12.207,-4.120,1.048,-3.337,-3.662,-1.933,-2.360,-0.478,11.607,9.013


In [26]:
def get_sequences(data, labels):
    sequences = []
    label_idx = 0
    for subject in range(0, len(data), 256):
        sequence_features = data[subject:subject+256]
        label = labels['label'][label_idx]
        label_idx += 1
        sequences.append((sequence_features, label))
        
    return sequences

ValueError: Found input variables with inconsistent numbers of samples: [2304000, 9000]

In [45]:
sequences = get_sequences(X, y)

In [46]:
len(sequences)

9000

In [47]:
random.shuffle(sequences)

In [72]:
train_sequences = sequences[0:int(len(sequences) * 0.75)]

In [144]:
test_sequences = sequences[int(len(sequences) * 0.75):]

In [145]:
len(train_sequences), len(test_sequences)

(6750, 2250)

In [89]:
zeros = 0
ones = 0
for data, label in train_sequences:
    if label == 0:
        zeros += 1
    elif label == 1:
        ones += 1
print(zeros, ones)

4478 2272


In [154]:
train_sequences[0]

(             2       3       4       5       6       7       8       9     10  \
 716288   2.665   0.285   5.198   5.870   3.123   3.418  -0.509   3.672  1.546   
 716289   3.153   1.261   6.175   4.893   1.658   1.953  -1.485   2.207  1.058   
 716290   3.153   3.215   4.710   4.405  -0.295   0.000  -2.950   0.743 -0.407   
 716291   2.177   4.679   2.268   4.893  -2.248  -2.441  -4.903  -0.722 -1.383   
 716292  -0.753   4.191  -0.661   3.916  -4.201  -4.395  -6.368  -2.187 -3.337   
 ...        ...     ...     ...     ...     ...     ...     ...     ...    ...   
 716539  -1.729  -8.993  -5.544 -24.404  -4.690  -8.789  -5.880  -9.511 -3.825   
 716540  -6.124  -9.969  -7.985 -27.334  -7.619 -11.230  -8.321 -11.464 -6.755   
 716541 -10.030 -12.410  -9.939 -29.775 -10.061 -12.207  -9.786 -12.441 -8.708   
 716542 -12.960 -16.317 -12.380 -32.216 -11.525 -12.695 -10.763 -12.929 -9.196   
 716543 -14.425 -20.711 -15.310 -35.146 -12.014 -13.184 -11.251 -13.906 -9.196   
 
             1

In [75]:
class SurfaceDataset(Dataset):
    
    def __init__(self, sequences):
        self.sequences = sequences
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        return dict(sequence=torch.Tensor(sequence.to_numpy()),
                   label=torch.Tensor([label]).long())

In [76]:
class SurfaceDataModule(pl.LightningDataModule):
    
    def __init__(self, train_sequences, test_sequences, batch_size):
        super().__init__()
        self.train_sequences = train_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size
        
    def setup(self, stage=None):
        self.train_dataset = SurfaceDataset(self.train_sequences)
        self.test_dataset = SurfaceDataset(self.test_sequences)
        return self.train_dataset
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, 
            batch_size = self.batch_size,
            shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset, 
            batch_size = self.batch_size,
            shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset, 
            batch_size = self.batch_size,
            shuffle=False)

In [77]:
N_EPOCHS = 250
BATCH_SIZE = 64
data_module = SurfaceDataModule(train_sequences, test_sequences, BATCH_SIZE)

# MODEL

In [78]:
class SequenceModel(nn.Module):
    
    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        
        super().__init__()
        
        self.n_hidden = n_hidden
        
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.75)
        
        self.classifier = nn.Linear(n_hidden, n_classes)
    
    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        out = hidden[-1]
        return self.classifier(out)

In [79]:
class SurfacePredictor(pl.LightningModule):
    
    def __init__(self, n_features: int, n_classes: int):
        super().__init__()
        self.model = SequenceModel(n_features=n_features, n_classes=n_classes)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x, labels=None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            #print(output.shape, '\n', labels.shape)
            loss = self.criterion(output, labels.squeeze())
        return loss, output
    
    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, output = self(sequences, labels)
        predictions = torch.argmax(output, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}
    
    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, output = self(sequences, labels)
        predictions = torch.argmax(output, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}
    
    def test_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, output = self(sequences, labels)
        predictions = torch.argmax(output, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.log("test_accuracy", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.0001)

In [80]:
model = SurfacePredictor(
    n_features=len(FEATURE_COLUMS), 
    n_classes=len(label_encoder.classes_))

In [81]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

Reusing TensorBoard on port 6006 (pid 16596), started 1 day, 13:23:56 ago. (Use '!kill 16596' to kill it.)

In [82]:
checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="best-checkpoint",
        save_top_k=1,
        verbose=True,
        monitor="val_loss",
        mode="min")

logger = TensorBoardLogger("lightning_logs", name="surface")

trainer = pl.Trainer(
        logger=logger,
        callbacks=[checkpoint_callback],
        max_epochs=N_EPOCHS,
        gpus=1,
        progress_bar_refresh_rate=30
)

C:\Users\Kuban\.conda\envs\master\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:446: UserWarning: Checkpoint directory checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [83]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | SequenceModel    | 1.4 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.532     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\Kuban\.conda\envs\master\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Kuban\.conda\envs\master\lib\site-packages\deprecate\deprecation.py:115: LightningDeprecationWarning: The `accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.functional.classification.accuracy.accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
Global seed set to 42
C:\Users\Kuban\.conda\envs\master\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this mach

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch 247, global step 26287: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 248, global step 26393: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 249, global step 26499: val_loss was not in top 1


In [87]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.7408888936042786, 'test_loss': 0.5501397252082825}
--------------------------------------------------------------------------------


[{'test_loss': 0.5501397252082825, 'test_accuracy': 0.7408888936042786}]

In [124]:
labels, test_dataset = pd.read_csv('data_9.csv', index_col=False), pd.read_csv("labels_9.csv", index_col=False)

In [93]:
labels['0']

0      c
1      c
2      c
3      c
4      c
      ..
495    c
496    c
497    c
498    c
499    c
Name: 0, Length: 500, dtype: object

In [125]:
encoded_labels = label_encoder.fit_transform(labels['0'])

In [95]:
encoded_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [132]:
encoded_labels = [1 for x in range(len(encoded_labels))]
labels['label'] = encoded_labels

In [134]:
sequences_test = get_sequences(test_dataset, labels)

In [128]:
COLUMS = test_dataset.columns.tolist()[2:-1]
COLUMS

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64']

In [129]:
test_dataset = test_dataset[COLUMS]

In [130]:
test_dataset

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,5.422,-0.081,4.028,-3.296,7.355,4.079,5.686,0.285,7.222,-1.373,...,0.804,0.824,2.431,-2.014,-2.757,-2.075,-3.174,-2.391,2.808,2.360
1,3.957,-1.546,2.563,-4.272,5.402,3.591,4.710,0.285,5.269,-0.397,...,0.804,1.312,1.943,-2.014,-2.268,-1.587,-3.174,-2.391,1.343,1.383
2,2.981,-2.035,1.587,-3.784,4.425,3.591,4.222,0.773,3.805,0.092,...,2.268,2.289,1.943,-1.038,0.173,-0.610,-2.197,-2.391,0.854,0.895
3,2.004,-1.058,1.587,-0.854,3.937,4.079,4.222,1.261,3.316,0.092,...,4.710,4.242,1.943,0.916,2.614,0.854,-0.244,-1.902,1.343,1.872
4,2.492,-0.081,2.563,2.075,4.425,4.567,4.710,1.750,3.805,-0.397,...,6.175,6.195,2.431,2.869,4.567,1.831,0.732,-1.902,3.296,3.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127995,33.376,46.427,7.701,22.339,22.217,27.588,16.581,24.089,10.701,14.435,...,-15.615,-5.981,6.775,-8.748,-13.367,-7.467,-12.207,-5.168,-0.478,-0.346
127996,35.817,49.845,7.701,21.851,23.193,28.564,16.581,25.065,10.213,14.435,...,-19.033,-6.470,6.775,-9.725,-14.832,-8.443,-13.672,-5.168,-0.966,-0.834
127997,37.771,50.334,6.724,19.897,23.193,28.564,16.093,24.577,9.725,12.970,...,-24.404,-7.446,6.287,-11.190,-16.785,-9.420,-15.625,-5.656,-1.455,-1.811
127998,39.724,50.334,6.236,17.944,23.193,28.076,16.093,24.089,8.748,10.529,...,-32.216,-9.888,6.775,-14.119,-20.691,-11.861,-19.531,-7.121,-1.943,-1.811


In [155]:
test_sequences[0]

(             2      3       4      5       6      7      8      9      10  \
 382976  -1.567 -2.696  -8.555 -2.350  -3.428 -0.702 -0.102  1.017  -3.052   
 382977  -6.449 -4.649  -8.555 -4.303  -2.452 -0.702  0.387  0.041  -3.052   
 382978  -9.379 -5.625  -9.043 -5.768  -1.475 -1.190  0.875 -0.448  -2.563   
 382979 -10.356 -6.602 -10.508 -6.256  -1.963 -1.678  0.387 -1.424  -2.075   
 382980 -11.332 -8.555 -13.438 -7.233  -3.428 -3.143 -1.078 -3.377  -2.563   
 ...        ...    ...     ...    ...     ...    ...    ...    ...     ...   
 383227   9.176  1.699   7.558 -5.280   8.291  3.204  4.293 -1.424   8.179   
 383228  12.105  3.652   9.511 -4.303   8.291  3.204  4.781 -0.936   8.179   
 383229  12.594  6.093  11.953 -2.350   9.267  4.669  5.758  1.017   9.155   
 383230  11.617  8.046  13.906  1.068  11.220  6.134  6.734  3.459  10.132   
 383231   9.664  8.535  13.418  4.486  12.197  7.599  7.711  5.900  10.620   
 
            11  ...      56      57     58      59      60    

In [142]:
label_encoder.classes_


array(['a', 'c'], dtype=object)

In [156]:
len(test_sequences)

2250

In [157]:
test_dataset = SurfaceDataset(test_sequences)

In [160]:
trained_model = SurfacePredictor.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path,
    n_features=64,
    n_classes=len(label_encoder.classes_))
trained_model.freeze()

In [163]:
predictions, labels = [], []
for item in test_dataset:
    sequence = item["sequence"]
    label = item["label"]
    _, output = trained_model(sequence.unsqueeze(dim=0))
    prediction = torch.argmax(output, dim=1)
    predictions.append(prediction.item())
    labels.append(label.item())

In [166]:
print(classification_report(labels, predictions, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

           a       0.81      0.80      0.81      1504
           c       0.61      0.61      0.61       746

    accuracy                           0.74      2250
   macro avg       0.71      0.71      0.71      2250
weighted avg       0.74      0.74      0.74      2250



In [168]:
corr = X.corr()

In [172]:
X['label'] = y['label']

In [177]:
corr = X.corr(method='pearson')

In [179]:
corr.style.background_gradient(cmap='coolwarm')

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,label
2,1.000000,0.811085,0.570559,0.550934,0.741862,0.746638,0.609771,0.570749,0.641803,0.405579,0.458384,0.356920,0.421853,0.220559,0.265970,0.014220,0.229381,0.190924,0.115660,0.063962,-0.032294,-0.063592,-0.040041,-0.071552,-0.119242,0.003205,0.037611,-0.068130,-0.071357,-0.036364,-0.021280,0.770569,0.703833,0.683486,0.624025,0.597595,0.408937,0.367686,0.797704,0.393890,0.466431,0.207748,0.280354,0.599717,0.625815,0.089424,0.149857,0.727227,0.073013,0.020281,0.013816,-0.019420,0.174045,0.158750,-0.021783,-0.028213,0.424813,-0.070604,-0.017562,-0.069735,-0.067007,-0.078330,0.324663,0.501648,-0.037057
3,0.811085,1.000000,0.592989,0.680439,0.813473,0.864074,0.687674,0.685825,0.688615,0.521357,0.468652,0.438266,0.446152,0.300770,0.264572,0.031868,0.218094,0.269735,0.102714,0.119887,-0.044850,-0.038362,-0.054247,-0.049559,-0.120742,0.036543,0.032319,-0.065589,-0.077585,-0.023680,-0.019428,0.857302,0.759709,0.814897,0.664662,0.728928,0.420232,0.474726,0.897868,0.516214,0.486869,0.294435,0.274361,0.704493,0.691719,0.151244,0.140151,0.819502,0.061813,0.068566,0.004314,0.008443,0.179895,0.222414,-0.021993,-0.010101,0.489234,-0.068502,-0.008152,-0.063149,-0.074382,-0.077234,0.348297,0.529456,-0.058272
4,0.570559,0.592989,1.000000,0.552949,0.636168,0.601927,0.538199,0.494223,0.652274,0.435103,0.652263,0.336968,0.457022,0.358410,0.553968,0.255427,0.427045,0.248860,0.373495,0.208812,0.143597,0.042938,0.152340,0.073771,0.024903,0.177735,0.292201,0.071181,0.097223,0.136797,0.169254,0.566245,0.598772,0.505864,0.683492,0.535889,0.641960,0.461916,0.605851,0.388383,0.598867,0.305435,0.548625,0.512025,0.584709,0.261609,0.434415,0.601468,0.290547,0.133064,0.239765,0.134719,0.293363,0.193254,0.191321,0.146212,0.392930,0.100780,0.148514,0.049314,0.081880,0.031945,0.458419,0.642711,-0.029867
5,0.550934,0.680439,0.552949,1.000000,0.612376,0.702614,0.563502,0.656702,0.551816,0.681572,0.480412,0.448067,0.389283,0.576976,0.405494,0.296650,0.277366,0.455179,0.252533,0.381268,0.077726,0.137006,0.100552,0.171872,0.041702,0.295737,0.217088,0.122869,0.104895,0.194338,0.176815,0.585174,0.535318,0.677826,0.556908,0.756305,0.484022,0.709328,0.639579,0.597529,0.450229,0.525464,0.368197,0.616691,0.546853,0.435575,0.318301,0.629184,0.178693,0.290588,0.171485,0.245279,0.217339,0.327233,0.168593,0.214184,0.414959,0.117441,0.174204,0.111313,0.077273,0.041485,0.386126,0.525353,-0.018351
6,0.741862,0.813473,0.636168,0.612376,1.000000,0.876023,0.801470,0.713209,0.832491,0.519083,0.612278,0.518439,0.612371,0.302974,0.386512,0.118731,0.355349,0.278810,0.197547,0.115091,0.026777,-0.014050,-0.010851,-0.045854,-0.110873,0.042154,0.080554,-0.048726,-0.056946,-0.007952,0.007078,0.723826,0.758661,0.709701,0.772981,0.697542,0.507894,0.452874,0.831159,0.538022,0.652505,0.290302,0.392333,0.778147,0.836526,0.152930,0.231859,0.868845,0.137283,0.053119,0.064289,0.003837,0.309892,0.249349,0.017701,-0.010971,0.607652,-0.060156,0.053655,-0.079796,-0.056176,-0.056748,0.370625,0.554794,-0.036780
7,0.746638,0.864074,0.601927,0.702614,0.876023,1.000000,0.844558,0.806176,0.802777,0.627827,0.562160,0.580349,0.605523,0.366776,0.336840,0.128445,0.306239,0.365448,0.152628,0.168786,-0.017307,-0.012214,-0.039078,-0.030255,-0.125148,0.063867,0.062090,-0.058219,-0.074761,-0.010258,-0.009401,0.744268,0.708751,0.750864,0.729119,0.801452,0.472773,0.534442,0.850156,0.648642,0.610920,0.361019,0.340886,0.853883,0.838084,0.204498,0.197432,0.895941,0.087012,0.099924,0.031302,0.029418,0.265008,0.304742,-0.004107,0.003196,0.642439,-0.060457,0.008454,-0.071909,-0.077663,-0.078579,0.350862,0.538808,-0.046522
8,0.609771,0.687674,0.538199,0.563502,0.801470,0.844558,1.000000,0.745464,0.806865,0.535944,0.565569,0.622744,0.701048,0.286084,0.324841,0.143949,0.348469,0.327826,0.153726,0.115945,0.011662,-0.020528,-0.044721,-0.064842,-